### MLE

In [1]:
import pytest

In [ ]:
def test_raise():
    with pytest.raises(ValueError) as e:
        raise_exception()

In [ ]:
l = {**d1, **d2}

In [ ]:
RecipeNutritionInformation = 

In [2]:
from typing import TypedDict

In [3]:
class NutritionInformation(TypedDict):
    value: int
    unit: str

In [4]:
class RecipeNutritionInformation(TypedDict):
    recipes_used: int
    calories: NutritionInformation
    carbs: NutritionInformation

In [ ]:
is_string()

In [ ]:
def test_raise():
    with pytest.raises(ValueError) as e:
        raise_exception()

In [ ]:
step 1: number of partrition
step 2: 

In [ ]:
docker pull redis

In [5]:
from typing import TypedDict

In [6]:
class NutritionInformation(TypedDict):
    value: int
    unit: str

In [7]:
class RecipeNutritionInformation(TypedDict):
    recipes_used: int
    calories: NutritionInformation
    carbs: NutritionInformation

### Engineering

In [ ]:
int i = 0;

In [ ]:
#include <iostream>

In [ ]:
for (; i < 10; i++) {
    std::cout << "i: " < i;
}

In [ ]:
int main() {
    int numbers[] = {1, 2, 3, 4, 5};
    
    for (int i; i <= 3; i++) {
        std::cout << numbers[i];
    }
    
    return 0;
}

In [ ]:
for (int x: xs) {
    std::cout << "value: " << x << std::endl;
}

In [ ]:
api server, control manager, ectd, scheduler

In [10]:
import torch
from torch import nn
import torch.nn.functional as F

In [11]:
def compute_forward_pass_using_data_parallelism(
    model, input,
    device_ids, output_id):
    models = nn.parallel.replicate(model, device_id)
    inputs = nn.parallel.scatter(input, device_id)
    
    logit = nn.parallel.parallel_apply(model, input)
    logits = nn.parallel.gather(logit, output_id)
    
    return logits

In [12]:
n_microbatches = 4

In [13]:
n_partritions = 3

In [14]:
n_clock_cycles = n_microbatches + n_partritions - 1

In [16]:
for clock_idx in range(n_clock_cycles):
    start_partrition = max(clock_idx+1-n_microbatches, 0)
    end_partrition = min(clock_idx+1, n_partritions)
    
    results = []
    for partrition_idx in range(start_partrition, end_partrition):
        microbatch_idx = clock_idx-partrition_idx
        results.append((microbatch_idx, clock_idx))
    
    print(results)

[(0, 0)]
[(1, 1), (0, 1)]
[(2, 2), (1, 2), (0, 2)]
[(3, 3), (2, 3), (1, 3)]
[(3, 4), (2, 4)]
[(3, 5)]


In [ ]:
api server, control manager, ectd, scheduler

In [ ]:
grid > thread block > thread

In [ ]:
global memory > shared memory > register memory

In [ ]:
step 1: cpu memory
step 2: gpu memory
step 3: copy from cpu to gpu
step 4: do the compuatation
step 5: copy the result back to cpu
step 6: dellocate GPU memory

In [ ]:
class VocabParallelEmbedding(nn.Module):
    def __init__(self, num_embeddings, embedding_dim):
        super().__init__()
        world_size = torch.distributed.get_world_size()
        self.num_embeddings = num_embeddings
        self.embedding_dim_per_partrition = embedding_dim // world_size
        
        self.weight = nn.Parameter(torch.empty(
            self.num_embeddings,
            self.embedding_dim_per_partrition
        ))
        self.vocab_start_idx, self.vocab_end_idx = self.get_vocab_range(
            self.embedding_dim_per_partrition
        )
    
    def get_vocab_range(self, embedding_dim_per_partrition):
        rank = torch.distributed.get_rank()
        start_idx = rank*embedding_dim_per_partrition
        end_idx = start_idx + embedding_dim_per_partrition
        return start_idx, end_idx
    
    def forward(self, tokens):
        masked = (self.vocab_start_idx > tokens) || (self.vocab_end_idx < tokens)
        tokens = tokens - self.vocab_start_idx
        tokens[masked] = 0.
        
        embeddings = F.embedding(tokens, self.weight)
        masked_idxs =
        embeddings[masked_idxs] = 0.
        
        return embeddings

In [18]:
def wait_stream(source_stream, target_stream):
    # ___ waits GPU
    if isinstance(target_stream, torch.cuda.Stream):
        if isinstance(source_stream, torch.cuda.Stream):
            # GPU waits for GPU
            source_stream.wait_stream(target_stream)
        else:
            # CPU waits for GPU
            target_stream.syncronous()

In [20]:
class Wait(torch.autograd.Function):
    @staticmethod
    def forward(ctx, prev_stream, next_stream, input):
        ctx.prev_stream = prev_stream
        ctx.next_stream = next_stream
        
        wait_stream(
            source_stream=next_stream,
            target_stream=prev_stream
        )
        
        return input

    @staticmethod
    def backward(ctx, grad_input):
        prev_stream = ctx.prev_stream
        next_stream = ctx.next_stream
        
        wait_stream(
            source_stream=prev_stream,
            target_stream=next_stream
        )
        
        grad_stream = (None, None)
        
        return grad_stream + grad_input

In [ ]:
partritioning
rank
device
local optimizer


In [21]:
import os

In [23]:
class MPU:
    def __init__(master_addr, master_port, backend):
        if not torch.distributed.is_initialized():
            rank = os.getenv("RANK")
            os.environ["MASTER_ADDR"] = str(master_addr)
            os.environ["MASTER_PORT"] = str(master_port)
            
            self.set_device(rank)
            
            torch.distributed.init_process_group(
                master_addr=master_addr,
                master_port=master_port,
                backend=backend
            )
    
    def set_device(self, rank):
        n_devices = torch.cuda.device_count()
        
        if n_devices > 0:
            device = rank % n_devices
            torch.distributed.set_device(device)

In [28]:
class f(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        return input

    @staticmethod
    def backward(ctx, grad_input):
        torch.distributed.all_reduce(grad_input)
        return grad_input

In [29]:
class g(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        world_size = torch.distributed.get_world_size()
        inputs = [torch.empty_like(input) for _ in range(world_size)]
        torch.distributed.all_gather(inputs, input)
        inputs = torch.cat(inputs, dim=-1)
        return inputs
    
    @staticmethod
    def backward(ctx, grad_input):
        rank = torch.distributed.get_rank()
        world_size = torch.distributed.get_world_size()
        
        dim_size = grad_input.shape[-1]
        dim_size_per_partrition = dim_size // world_size
        
        grad_chunks = torch.split(grad_input, dim_size_per_partrition, dim=-1)
        return grad_chunks[rank]

In [25]:
class ColumnParallelLinear(nn.Module):
    def __init__(self, input_size, output_size, world_size):
        super().__init__()
        self.input_size = input_size
        self.output_size_per_partrition = output_size // world_size
        
        self.weight = nn.Parameter(torch.empty(
            self.output_size_per_partrition,
            self.input_size
        ))
        self.bias = nn.Parameter(torch.empty(
            self.output_size_per_partrition
        ))
    
    def forward(self, input):
        input_parallel = f.apply(input)
        output_parallel = F.linear(
            input_parallel,
            self.weight,
            self.bias
        )
        outputs = g.apply(output_parallel)
        return outputs

In [ ]:
int* p_number{};

In [ ]:
def create_continuous_memory(memory_size):
    n_elements = memory_size // 4
    return torch.randn(n_elements, dtype=torch.float32)

In [ ]:
int x = y;

In [ ]:
int* p_x{&y};

In [31]:
from einops import pack

In [ ]:
stacked_images = pack(image_rgb, image_depth, "b c *")

In [ ]:
tokens = model.to_tokens(text)

In [ ]:
logits = model(tokens)

In [ ]:
log_probs = F.log_softmax(logits, dim=-1)

In [ ]:
target_tokens = tokens[:, 1:]

In [ ]:
-log_probs[target_tokens]